### Ref1: https://rateslib.readthedocs.io/en/stable/z_swpm.html
### Ref2: https://quant.stackexchange.com/a/49585

### Default SOFR

![Default SOFR](sofr_n_swap/default_sofr.png)

### Swap

![Swap](sofr_n_swap/swap.png)

In [1]:
import pandas as pd
from rateslib import *

In [2]:
def get_pv_results(bump=0.0):
    data = pd.DataFrame({
    "Term": ["1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M", "12M", "18M", "2Y", "3Y", "4Y"],
    "Rate": [5.30111, 5.30424, 5.30657, 5.31100, 5.34800, 5.38025, 5.40915, 5.43078, 5.44235, 5.44950, 5.44878, 5.44100, 5.42730, 5.40747, 5.3839, 5.09195, 4.85785, 4.51845, 4.31705],
    })
    
    data["Rate"] += bump

    
    data["Termination"] = [add_tenor(dt(2023, 8, 21), _, "F", "nyc") for _ in data["Term"]]
    
    sofr = Curve(
        id="sofr",
        convention="Act360",
        calendar="nyc",
        modifier="MF",
        interpolation="log_linear",
        nodes={
            **{dt(2023, 8, 17): 1.0},  # <- this is today's DF,
            **{_: 1.0 for _ in data["Termination"]},
        }
    )

    sofr_args = dict(effective=dt(2023, 8, 21), spec="usd_irs", curves="sofr")

    solver = Solver(
        curves=[sofr],
        instruments=[IRS(termination=_, **sofr_args) for _ in data["Termination"]],
        s=data["Rate"],
        instrument_labels=data["Term"],
        id="us_rates",
    )

    data["DF"] = [float(sofr[_]) for _ in data["Termination"]]
    
    irs = IRS(
        effective=dt(2023, 11, 21),
        termination=dt(2025, 2, 21),
        notional=-100e6,
        fixed_rate=5.40,
        curves="sofr",
        spec="usd_irs",
    )
    
    npv = irs.npv(solver=solver)
    dv01 = irs.delta(solver=solver).sum()
    pv01 = irs.analytic_delta(curve=sofr)
    return npv, dv01, pv01

In [3]:
npv, dv01, pv01 = get_pv_results(bump=0.0)

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.116743265440467e-17, `time`: 0.0718s


### PV

The net present value (PV) of a vanilla IRS can be computed by determining the PV of each fixed leg and floating leg separately and summing. 

For pricing a mid-market IRS the underlying principle is that the two legs must have the same value initially.

#### Fixed Leg

Calculating the fixed leg requires discounting all of the known cashflows by an appropriate discount factor:

$$P_{fixed} = N R \sum_{i=1}^{n_1} d_i v_i$$

$N$ = notional

$R$ = fixed rate

$n_1$ = the number of payments

$d_i$ = decimalised day count fraction of the accrual in the i'th period

$v_i$ = discount factor associated with the payment date of the i'th period

#### Floating Leg

Calculating the floating leg is a similar process replacing the fixed rate with __forecast index rates__(projection curves):

$$P_{float} = N \sum_{i=1}^{n1} r_j d_j v_j$$

$n_2$ = the number of payments of the floating leg

$r_j$ = forecast -IBOR index rates of the appropriate currency

#### PV

$$P_{IRS} = P_{fixed} - P_{float}$$


$$P_{IRS} = N R \sum_{i=1}^{n_1} d_i v_i - N \sum_{i=1}^{n1} r_j d_j v_j$$

In [4]:
print("PV", npv.real)

PV 456622.09860395174


### PV01
Or __Analytic PV01__:when you change the value of the fixed coupon by 1bp and evaluate the impact on the IRS

$$P = R \sum_{i=1}^{n_i} d_i v_i - \sum_{j=1}^{n_j} r_j d_j v_j$$

$$\frac{\partial P}{\partial R} = \underbrace{ \sum_{i=1}^{n_i} d_i v_i }_{analytic \ fixed \ leg}$$

$d$ = day fraction

$v$ = the discount factors

$r$ = the floating rates

The notation here is a bit mangled because the i's and the j's might refer to schedules of different frequencies. 


In [5]:
pv01

<Dual: -11,896.008577, ('sofr10', 'sofr16', 'sofr17', 'sofr9'), [ -176.95178153 -9937.61435225  -229.09616173 -2378.60529106]>

### DV01
Or Real Portfolio PV01 / Delta:

In [6]:
print("DV01", dv01)

DV01 local_ccy  display_ccy
usd        usd           -11879.936805
dtype: float64


### DV01 - Central Finite Difference

Assume the PnL on a swap is almost its linear pnl plus its convexity:

$$\Delta P (\Delta r) \approx \frac{\partial P}{\partial r} \Delta r  + \frac{1}{2} \frac{{\partial}^2 P}{\partial r^2} \Delta r^2$$

Then bumping by +1bp and -1bp, dividing by 2 eliminates the convexity element and very accurately approximates the real PV01:

$$\frac{\Delta P (+1bp) - \Delta P (-1bp)}{2} = \frac{\partial P}{\partial r}$$

In [7]:
one_bp = 0.01

In [8]:
npv_plus_1bp, _, _ = get_pv_results(bump=one_bp)
npv_plus_1bp_val = npv_plus_1bp.real

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.108013716766087e-17, `time`: 0.0743s


In [9]:
npv_minus_1bp, _, _ = get_pv_results(bump=-one_bp)
npv_minus_1bp_val = npv_minus_1bp.real

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.1254963583773556e-17, `time`: 0.0773s


In [10]:
dv01_finite_diff = (npv_plus_1bp_val - npv_minus_1bp_val) / 2.0
print(dv01_finite_diff)

-11879.93697656691


### DV01 - Single Bumped Curve
Another common method of calculation is to use a single bumped curve by, say, $\frac{1}{100}$ th of a bp, and scale the result by 100. Although less accurate, since the convexity is marginalised and not eliminated, the calculation is twice as fast, for example:

$100 \Delta P (+\frac{1}{100}bp) = \frac{\partial P}{\partial r} + \frac{1}{200} \frac{{\partial}^2 P}{\partial r^2} $

In [11]:
npv_bump, _, _ = get_pv_results(bump=one_bp*0.01)
npv_bump_val = npv_bump.real

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.116670055661461e-17, `time`: 0.0738s


In [12]:
dv01_single_bump = (npv.real - npv_bump_val) * 100.0
print(dv01_single_bump)

11879.920919146389


## Extra Info
Historically IRSs were valued using discount factors derived from the same curve used to forecast the -IBOR rates. This has been called "self-discounted". Some early literature described some incoherence introduced by that approach and multiple banks were using different techniques to reduce them. It became more apparent with the 2007–2012 global financial crisis that the approach was not appropriate, and alignment towards discount factors associated with __physical collateral__ of the IRSs was needed.

Post crisis, to accommodate credit risk, the now-standard pricing approach is the multi-curve framework where forecast -IBOR rates and discount factors exhibit disparity. Note that the economic pricing principle is unchanged: __leg values are still identical at initiation.__

Here, overnight index swap (OIS) rates are typically used to __derive discount factors__, since that index is the standard inclusion on Credit Support Annexes (CSAs) to determine the rate of interest payable on collateral for IRS contracts. 

As regards the rates forecast, since the basis spread between LIBOR rates of different maturities widened during the crisis, forecast curves are generally constructed for each LIBOR tenor used in floating rate derivative legs.[4]

Under both frameworks, the following apply. 

1. Maturities for which rates are solved directly are referred to as __"pillar points"__, these correspond to the input-instrument maturities; other rates are interpolated, often using Hermitic splines. 

2. The objective function: prices must be "exactly" returned, as described. 

3. The penalty function will weigh: that forward rates are positive (to be arbitrage free) and curve "smoothness"; both, in turn, a function of the interpolation method. [7] [8] [9] 

4. The initial estimate: usually, the most recently solved curve set. 
5. All that need be stored are the solved spot rates for the pillar points, and the interpolation rule.